In [8]:
# =========================================================
# FINAL 1-CELL LEADERBOARD PIPELINE (STABLE ≥0.70)
# =========================================================
import pandas as pd
import numpy as np
from pathlib import Path
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

SEED = 42
np.random.seed(SEED)

# =========================================================
# LOAD
# =========================================================
df = pd.read_csv("merged_libur_cuaca_ispu_ndvi.csv", sep=";")
df["tanggal"] = pd.to_datetime(df["tanggal"], dayfirst=True)
df = df.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

df = df.drop(columns=["max","parameter_pencemar_kritis","time","id","stasiun"], errors="ignore")

LABEL_MAP = {"BAIK":0,"SEDANG":1,"TIDAK SEHAT":2}
df = df[df["kategori"].notna()].copy()
df["target"] = df["kategori"].map(LABEL_MAP).astype(int)

# =========================================================
# REINDEX HARIAN
# =========================================================
def reindex_daily(g):
    idx = pd.date_range(g["tanggal"].min(), g["tanggal"].max(), freq="D")
    g = g.set_index("tanggal").reindex(idx)
    g["lokasi_clean"] = g["lokasi_clean"].iloc[0]
    return g.reset_index().rename(columns={"index":"tanggal"})

df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)

# =========================================================
# PM2.5 IMPUTATION
# =========================================================
median_pm25 = df[df["tanggal"] >= "2021-01-01"].groupby("lokasi_clean")["pm_duakomalima"].median()
df["pm_duakomalima"] = df["pm_duakomalima"].fillna(df["lokasi_clean"].map(median_pm25))
df = df[df["target"].notna()].copy()

# =========================================================
# TIME FEATURES
# =========================================================
df["month"] = df["tanggal"].dt.month
df["dayofyear"] = df["tanggal"].dt.dayofyear
df["dayofweek"] = df["tanggal"].dt.dayofweek

df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
df["month_cos"] = np.cos(2*np.pi*df["month"]/12)
df["doy_sin"] = np.sin(2*np.pi*df["dayofyear"]/365)
df["doy_cos"] = np.cos(2*np.pi*df["dayofyear"]/365)
df["dow_sin"] = np.sin(2*np.pi*df["dayofweek"]/7)
df["dow_cos"] = np.cos(2*np.pi*df["dayofweek"]/7)

# =========================================================
# POLLUTANTS
# =========================================================
POLLUTANT_COLS = [
    "pm_sepuluh","pm_duakomalima","ozon",
    "nitrogen_dioksida","sulfur_dioksida","karbon_monoksida"
]

# =========================================================
# BUILD MULTI-HORIZON FORECAST DATA
# =========================================================
def build_forecast_df(df_source, H=30):
    rows = []
    for loc, g in df_source.groupby("lokasi_clean"):
        g = g.sort_values("tanggal").reset_index(drop=True)

        for col in POLLUTANT_COLS:
            g[f"{col}_lag1"] = g[col].shift(0)
            g[f"{col}_lag2"] = g[col].shift(1)
            g[f"{col}_lag3"] = g[col].shift(2)

        for h in range(1, H+1):
            g_future = g.shift(-h)

            temp = pd.DataFrame({
                "lokasi_clean": loc,
                "tanggal": g["tanggal"],
                "horizon": h,
                "month": g["tanggal"].dt.month,
                "dayofyear": g["tanggal"].dt.dayofyear,
                "dayofweek": g["tanggal"].dt.dayofweek,
            })

            for col in POLLUTANT_COLS:
                temp[f"{col}_lag1"] = g[f"{col}_lag1"]
                temp[f"{col}_lag2"] = g[f"{col}_lag2"]
                temp[f"{col}_lag3"] = g[f"{col}_lag3"]
                temp[f"target_{col}"] = g_future[col]

            rows.append(temp)

    return pd.concat(rows, ignore_index=True).dropna().reset_index(drop=True)

df_model = df.copy()
df_forecast = build_forecast_df(df_model)

# =========================================================
# ENCODE LOCATION
# =========================================================
le_loc = LabelEncoder()
df_model["lokasi_enc"] = le_loc.fit_transform(df_model["lokasi_clean"])
df_forecast["lokasi_enc"] = le_loc.transform(df_forecast["lokasi_clean"])

# =========================================================
# TIME SPLIT
# =========================================================
SPLIT_DATE = "2024-01-01"
df_forecast_train = df_forecast[df_forecast["tanggal"] < SPLIT_DATE]
df_forecast_val   = df_forecast[df_forecast["tanggal"] >= SPLIT_DATE]

# =========================================================
# TRAIN FORECAST MODELS (NO NOISE, STABLE)
# =========================================================
FEATURES_F = (
    [f"{c}_lag1" for c in POLLUTANT_COLS] +
    [f"{c}_lag2" for c in POLLUTANT_COLS] +
    [f"{c}_lag3" for c in POLLUTANT_COLS] +
    ["month","dayofyear","dayofweek","lokasi_enc"]
)

pollutant_models = {col:{} for col in POLLUTANT_COLS}

for col in POLLUTANT_COLS:
    for h in range(1,31):

        df_h_train = df_forecast_train[df_forecast_train["horizon"]==h]
        df_h_val   = df_forecast_val[df_forecast_val["horizon"]==h]
        if len(df_h_val) < 50:
            continue

        model = lgb.train(
            dict(objective="regression", metric="l2", learning_rate=0.05, seed=SEED),
            lgb.Dataset(df_h_train[FEATURES_F], label=df_h_train[f"target_{col}"]),
            num_boost_round=2000,
            valid_sets=[lgb.Dataset(df_h_val[FEATURES_F], label=df_h_val[f"target_{col}"])],
            callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
        )

        pollutant_models[col][h] = model

# =========================================================
# BUILD ISPU DATA
# =========================================================
for col in POLLUTANT_COLS:
    for lag in [1,2,3]:
        df_model[f"{col}_lag{lag}"] = df_model.groupby("lokasi_clean")[col].shift(lag)

df_model = df_model.dropna().reset_index(drop=True)

train_df = df_model[df_model["tanggal"] < SPLIT_DATE]
val_df   = df_model[df_model["tanggal"] >= SPLIT_DATE]

ISPU_FEATURES = (
    POLLUTANT_COLS +
    [f"{c}_lag{l}" for c in POLLUTANT_COLS for l in [1,2,3]] +
    ["month_sin","month_cos","doy_sin","doy_cos","dow_sin","dow_cos","lokasi_enc"]
)

X_train, y_train = train_df[ISPU_FEATURES], train_df["target"]
X_val,   y_val   = val_df[ISPU_FEATURES],   val_df["target"]

# =========================================================
# TRAIN ISPU MODEL
# =========================================================
ispu_model = lgb.train(
    dict(objective="multiclass", num_class=3, metric="multi_logloss", learning_rate=0.05, seed=SEED),
    lgb.Dataset(X_train, label=y_train),
    num_boost_round=2000,
    valid_sets=[lgb.Dataset(X_val, label=y_val)],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(50)]
)

# =========================================================
# THRESHOLD (SAFE)
# =========================================================
probs_val = ispu_model.predict(X_val)
best_t = np.array([0.45, 0.35, 0.25])

for c in range(3):
    best_f1 = 0
    for t in np.linspace(0.2,0.6,25):
        pred = probs_val.argmax(1)
        pred[probs_val[:,c] > t] = c
        f1 = f1_score(y_val, pred, average="macro")
        if f1 > best_f1:
            best_f1 = f1
            best_t[c] = t

print("Best threshold:", best_t)

# =========================================================
# FORECAST TEST (NO NOISE, SAFE ENSEMBLE)
# =========================================================
sub = pd.read_csv("sample_submission.csv")
sub["tanggal"] = pd.to_datetime(sub["id"].str.split("_").str[0])
sub["lokasi_clean"] = sub["id"].str.split("_").str[1]
sub["lokasi_enc"] = le_loc.transform(sub["lokasi_clean"])

last_dates = df_model.groupby("lokasi_clean")["tanggal"].max().to_dict()

forecast_rows = []

for loc in sub["lokasi_clean"].unique():

    hist = (
        df_model[df_model["lokasi_clean"]==loc]
        .sort_values("tanggal")
        .iloc[-3:][POLLUTANT_COLS]
        .values.tolist()
    )

    future_rows = sub[sub["lokasi_clean"]==loc].sort_values("tanggal")

    for _, row in future_rows.iterrows():

        tgl = row["tanggal"]
        h = max(1, min(30, (tgl - last_dates[loc]).days))

        feat = {
            "month": tgl.month,
            "dayofyear": tgl.dayofyear,
            "dayofweek": tgl.dayofweek,
            "lokasi_enc": row["lokasi_enc"],
        }

        for i, col in enumerate(POLLUTANT_COLS):
            feat[f"{col}_lag1"] = hist[-1][i]
            feat[f"{col}_lag2"] = hist[-2][i]
            feat[f"{col}_lag3"] = hist[-3][i]

        Xf = pd.DataFrame([feat])

        new_vals = []
        preds_pol = {}

        for i, col in enumerate(POLLUTANT_COLS):

            model = pollutant_models[col].get(h)
            pred_lgb = model.predict(Xf)[0] if model else hist[-1][i]
            pred_last = hist[-1][i]

            pred = 0.85*pred_lgb + 0.15*pred_last  # STABLE ENSEMBLE

            new_vals.append(pred)
            preds_pol[col] = pred

        hist.append(new_vals)
        hist.pop(0)

        feat.update(preds_pol)

        for i, col in enumerate(POLLUTANT_COLS):
            feat[f"{col}_lag1"] = hist[-1][i]
            feat[f"{col}_lag2"] = hist[-2][i]
            feat[f"{col}_lag3"] = hist[-3][i]

        feat["id"] = row["id"]
        forecast_rows.append(feat)

df_future = pd.DataFrame(forecast_rows)

# =========================================================
# FINAL PREDICTION
# =========================================================
results = []
for _, row in df_future.iterrows():

    Xi = pd.DataFrame([{
        **{col:row[col] for col in POLLUTANT_COLS},
        **{f"{col}_lag{l}":row[f"{col}_lag{l}"] for col in POLLUTANT_COLS for l in [1,2,3]},
        "month_sin":np.sin(2*np.pi*row["month"]/12),
        "month_cos":np.cos(2*np.pi*row["month"]/12),
        "doy_sin":np.sin(2*np.pi*row["dayofyear"]/365),
        "doy_cos":np.cos(2*np.pi*row["dayofyear"]/365),
        "dow_sin":np.sin(2*np.pi*row["dayofweek"]/7),
        "dow_cos":np.cos(2*np.pi*row["dayofweek"]/7),
        "lokasi_enc":row["lokasi_enc"],
    }])

    prob = ispu_model.predict(Xi)[0]
    pred = int(np.argmax(prob - best_t))
    results.append(pred)

INV_LABEL_MAP = {0:"BAIK",1:"SEDANG",2:"TIDAK SEHAT"}

pred_df = df_future[["id"]].copy()
pred_df["kategori"] = [INV_LABEL_MAP[i] for i in results]

sub_final = pd.read_csv("sample_submission.csv")
sub_final = sub_final.merge(pred_df, on="id", how="left")
sub_final.to_csv("submission_FINAL_STABLE.csv", index=False)

print("DONE — submission_FINAL_STABLE.csv")


C:\Users\USER\AppData\Local\Temp\ipykernel_10552\2987440144.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001513 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2447
[LightGBM] [Info] Number of data points in the train set: 12220, number of used features: 22
[LightGBM] [Info] Start training from score 55.207147
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[74]	valid_0's l2: 127.009
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001380 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2447
[LightGBM] [Info] Number of data points in the train set: 12220, number of used features: 22
[LightGBM] [Info] Start training from score 55.213121
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[62]	valid_0's l2: 149.476
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the over

In [ ]:
import pandas as pd

df = pd.read_csv("submission_FINAL_STABLE.csv", sep=";")  
df.to_csv("submission_final_STABLE", index=False)     

: 

In [10]:
pd.Series(results).value_counts(normalize=True)


1    0.879121
0    0.120879
Name: proportion, dtype: float64

In [11]:
print(best_t)


[0.48333333 0.2        0.25      ]
